In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch, ordered_channels_membrane, ordered_modules_membrane, strUch, ordered_channels_cathode
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02, runBasicWfAnaNP02Updating
from waffles.np02_utils.load_utils import open_processed

In [ ]:
dettype = "membrane"
# run = 39036

# dettype = "cathode"
run = 39033

detectors = None # None for all detectors


## Only change if necessary
datadir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107

channels = None if detectors is None else [ uch.channel for m, uch in dict_module_to_uniqch.items() if m in detectors ]

In [ ]:
from collections import Counter
import copy
import time
def remove_weird_signals(waveform: Waveform) -> bool:
    if np.any((waveform.adcs[0:500] > 16000) | (waveform.adcs[0:500] < 100) ):
        return False
    return True
    
nwaveforms = None
wfset = open_processed(run, dettype, datadir, channels, [endpoint], nwaveforms=nwaveforms, verbose=True, mergefiles=False)
wfset = WaveformSet.from_filtered_WaveformSet(wfset, remove_weird_signals, show_progress=True)
    

In [ ]:
runBasicWfAnaNP02(wfset, threshold=20, baselinefinish = 240, onlyoptimal=False, configyaml="")

In [ ]:
pretriggerfinish = 240
def remove_signals_pretrigger(waveform: Waveform, ptfinish:int) -> bool:
    if np.any(abs(waveform.adcs[:ptfinish] - waveform.analyses['std'].result['baseline']) > 50):
        return False
    return True

wfset = WaveformSet.from_filtered_WaveformSet(wfset, remove_signals_pretrigger, pretriggerfinish, show_progress=True)
    

In [ ]:
wfch = ChannelWsGrid.clusterize_waveform_set(wfset)

In [ ]:
print(wfch)
print(len(wfset.waveforms))


In [ ]:
# Definition for RMS:
quantile = [0.02, 0.98]

f = open(f"noise_results_file_{dettype}.csv", "w")
f.write("endpoint,channel,rms\n")
chlist = ordered_channels_membrane if dettype == "membrane" else ordered_channels_cathode
for ch in chlist:
    ep = endpoint
    if ch not in wfch[ep]:
        continue
    wfs = wfch[ep][ch]
    print(ep, ch, 'nwaveforms:', len(wfs.waveforms))
    adcsvals = np.ravel([ wf.adcs[:pretriggerfinish] - wf.analyses['std'].result['baseline'] for wf in wfs.waveforms ])
    qlow, qhigh = np.quantile(adcsvals, quantile)
    rms = np.std(adcsvals[ (adcsvals > qlow) & (adcsvals<qhigh) ])
    # rms = np.std(adcsvals)
    f.write(f"{ep},{ch},{rms:.2f}\n")
f.close()
    
    